In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import os

url = "https://talemecasting-next.com/talent?birth_year_min=&birth_year_max=&age_min=15&age_max=40&height_min=0&height_max=200&weight_min=0&weight_max=200&blood=&constellation=&keyword=&carrier="
response = requests.get(url)

In [32]:
soup = BeautifulSoup(response.text, 'lxml')
title = soup.title.string
print(title)

芸能人一覧 | キャスティング業務用データベース「タレメcasting NEXT」


In [33]:
link = soup.tbody.find_all("tr", class_="talent-data")
name = link[0].find("span", class_="fwb").string
height = link[0].find("td", class_="height").string
height = re.sub(r"\D", "", height)
weight = link[0].find("td", class_="weight").string
weight = re.sub(r"\D", "", weight)
link = link[0].find("img")["src"]
print(name, height, weight, link)

半野　雅 176 65 https://narrow.jp/img/upload/talent/M22-0485-211114.jpg


In [27]:
img = requests.get(link)
with open("img.jpg", "wb") as f:
    f.write(img.content)

In [60]:
ids = []
names = []
heights = []
weights = []
bmis = []
os.makedirs("talents_dataset", exist_ok=True)

In [ ]:
import time
first_time = True  # 既にtalents.csvにデータがある場合はFalseにする
last_save_idx = 0  # 最後に保存したID（途中で処理を中断した場合に使用）

for i in range(1, 34):
    url = f"https://talemecasting-next.com/talent?birth_year_min=&birth_year_max=&age_min=15&age_max=40&height_min=0&height_max=300&weight_min=0&weight_max=200&blood=&constellation=&keyword=&limit=100&carrier=&page={i}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    body = soup.tbody.find_all("tr", class_="talent-data")
    talents = body.copy()
    for j, talent in enumerate(talents):
        if (i - 1) * 100 + j > last_save_idx:
            ids.append((i - 1) * 100 + j)
            name = talent.find("span", class_="fwb").string.replace("\u3000", "")
            names.append(name)
            height = talent.find("td", class_="height").string
            height = int(re.sub(r"\D", "", height))
            heights.append(height)
            weight = talent.find("td", class_="weight").string
            weight = int(re.sub(r"\D", "", weight))
            weights.append(weight)
            bmi = weight / (height / 100) ** 2
            bmis.append(bmi)
            img_link = talent.find("img")["src"]
            img = requests.get(img_link)
            save_path = f"talents_dataset/{(i - 1) * 100 + j:04}.jpg"
            with open(save_path, "wb") as f:
                f.write(img.content)
            time.sleep(1)

    print(f"{i} pages done")
    df = pd.DataFrame(list(zip(ids, names, heights, weights, bmis)), columns=["id", "name", "height", "weight", "bmi"])
    if first_time:
        df.to_csv("talents_dataset/hoge.csv", index=False)
        first_time = False
    else:
        df.to_csv("talents_dataset/hoge.csv", mode="a", header=False, index=False)
    ids = []
    names = []
    heights = []
    weights = []
    bmis = []
    time.sleep(5)

In [ ]:
df = pd.DataFrame(list(zip(ids, names, heights, weights, bmis)), columns=["id", "name", "height", "weight", "bmi"])
df

In [54]:
df.to_csv("talents_dataset/talents.csv", mode="a", header=False, index=False)

上記の操作で作成したデータセットでの予測精度が非常に低かったため、追加のデータを取得する

In [4]:
url = f"https://talemecasting-next.com/talent?birth_year_min=&birth_year_max=&age_min=15&age_max=40&height_min=0&height_max=300&weight_min=0&weight_max=200&blood=&constellation=&keyword=&limit=100&carrier=&page="
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')
body = soup.tbody.find_all("tr", class_="talent-data")
talents = body.copy()
print(talents[0].find("p", class_="name"))
link = talents[0].find("p", class_="name").find("a").get("href")
print(link)

<p class="name">
<a href="/talent/121754"><span class="fwb">半野　雅</span><br/>
                                            ハンノ　マサシ</a>
</p>
/talent/121754


In [7]:
person_url = f"https://talemecasting-next.com{link}"
print(person_url)

https://talemecasting-next.com/talent/121754


In [11]:
person_url = f"https://talemecasting-next.com{link}"
p_response = requests.get(person_url)
p_soup = BeautifulSoup(p_response.text, 'lxml')
# print(p_soup)
p_body = p_soup.find_all("table", class_="profile")
print(list(p_body))
hoge = p_body[0].find_all("tr")
print(hoge)

[<table class="profile">
<tr>
<th>ジャンル</th>
<td>俳優、モデル</td>
</tr>
<tr>
<th>性別</th>
<td>男性</td>
</tr>
<tr>
<th>生年月日</th>
<td>1984年07月05日（38歳）</td>
</tr>
<tr>
<th>サイズ</th>
<td>身長/176cm　体重/65cm　バスト/--　ウエスト/--　ヒップ/--</td>
</tr>
<tr>
<th>血液型</th>
<td>A型</td>
</tr>
<tr>
<th>星座</th>
<td>かに座</td>
</tr>
</table>]
[<tr>
<th>ジャンル</th>
<td>俳優、モデル</td>
</tr>, <tr>
<th>性別</th>
<td>男性</td>
</tr>, <tr>
<th>生年月日</th>
<td>1984年07月05日（38歳）</td>
</tr>, <tr>
<th>サイズ</th>
<td>身長/176cm　体重/65cm　バスト/--　ウエスト/--　ヒップ/--</td>
</tr>, <tr>
<th>血液型</th>
<td>A型</td>
</tr>, <tr>
<th>星座</th>
<td>かに座</td>
</tr>]


In [16]:
print(str(hoge))
old = str(hoge).split("生年月日</th>\n<td>")[1].split("</td>")[0]
sex = str(hoge).split("性別</th>\n<td>")[1].split("</td>")[0]
print(old, sex)

[<tr>
<th>ジャンル</th>
<td>俳優、モデル</td>
</tr>, <tr>
<th>性別</th>
<td>男性</td>
</tr>, <tr>
<th>生年月日</th>
<td>1984年07月05日（38歳）</td>
</tr>, <tr>
<th>サイズ</th>
<td>身長/176cm　体重/65cm　バスト/--　ウエスト/--　ヒップ/--</td>
</tr>, <tr>
<th>血液型</th>
<td>A型</td>
</tr>, <tr>
<th>星座</th>
<td>かに座</td>
</tr>]
1984年07月05日（38歳） 男性


In [37]:
import time
ids, olds, sexs = [], [], []

for i in range(20, 21):
    url = f"https://talemecasting-next.com/talent?birth_year_min=&birth_year_max=&age_min=15&age_max=40&height_min=0&height_max=300&weight_min=0&weight_max=200&blood=&constellation=&keyword=&limit=100&carrier=&page={i}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    body = soup.tbody.find_all("tr", class_="talent-data")
    talents = body.copy()
    for j, talent in enumerate(talents):
        if (i - 1) * 100 + j <= 851:
            continue
        elif (i - 1) * 100 + j > 1966:
            break
        link = talent.find("p", class_="name").find("a").get("href")
        person_url = f"https://talemecasting-next.com{link}"
        p_response = requests.get(person_url)
        p_soup = BeautifulSoup(p_response.text, 'lxml')
        p_body = p_soup.find_all("table", class_="profile")
        try:
            p_info = p_body[0].find_all("tr")
            old = str(p_info).split("生年月日</th>\n<td>")[1].split("</td>")[0]
            sex = str(p_info).split("性別</th>\n<td>")[1].split("</td>")[0]
        except:
            old = None
            sex = None
        ids.append((i - 1) * 100 + j)
        olds.append(old)
        sexs.append(sex)
        print((i - 1) * 100 + j, old, sex)
    time.sleep(1.5)
    print(f"{i} pages done")

1900 1999年04月30日（23歳） 女性
1901 1985年09月03日（36歳） 男性
1902 1987年03月23日（35歳） 男性
1903 1991年05月11日（31歳） 男性
1904 1984年11月02日（37歳） 女性
1905 1984年09月28日（37歳） 男性
1906 1987年06月02日（35歳） 男性
1907 1982年06月06日（40歳） 男性
1908 1983年03月26日（39歳） 男性
1909 1990年06月26日（32歳） 男性
1910 1989年07月25日（32歳） 男性
1911 1984年09月05日（37歳） 男性
1912 1984年07月15日（38歳） 男性
1913 2005年04月19日（17歳） 女性
1914 1994年05月01日（28歳） 男性
1915 1987年12月02日（34歳） 男性
1916 1990年08月10日（31歳） 男性
1917 1990年06月15日（32歳） 男性
1918 1981年10月11日（40歳） 男性
1919 1982年03月13日（40歳） 女性
1920 1992年02月05日（30歳） 男性
1921 2002年12月26日（19歳） 女性
1922 1982年07月17日（40歳） 男性
1923 1995年05月15日（27歳） 男性
1924 1985年04月30日（37歳） 女性
1925 1993年09月14日（28歳） 女性
1926 1983年07月10日（39歳） 男性
1927 1990年11月24日（31歳） 男性
1928 1998年10月09日（23歳） 女性
1929 1999年08月11日（22歳） 男性
1930 1991年05月27日（31歳） 女性
1931 1995年11月17日（26歳） 男性
1932 1989年07月06日（33歳） 男性
1933 1982年03月10日（40歳） 男性
1934 1995年06月08日（27歳） 男性
1935 1984年07月20日（37歳） 男性
1936 1986年12月10日（35歳） 男性
1937 1990年09月02日（31歳） 男性
1938 1988年12月27日（33歳） 男性
1939 1995年07月09日（27歳） 男性


In [28]:
person_url

'https://talemecasting-next.com/talent/78039'

In [38]:
df = pd.DataFrame(list(zip(ids, olds, sexs)), columns=["id", "old", "sex"])
df.to_csv("talents_dataset/addition.csv", mode="a", index=False, header=False)

データの統合

In [88]:
import pandas as pd

df1 = pd.read_csv("talents_dataset/talents.csv")
df2 = pd.read_csv("talents_dataset/addition.csv")

In [89]:
for d in [df1, df2]:
    df_id = d["id"].tolist()
    pre = -1
    for i in df_id:
        if i != 1 + pre:
            print(i)
        pre = i

In [90]:
df = pd.concat([df1, df2.drop("id", axis=1)], axis="columns", join="inner")
df

,id,name,height,weight,bmi,old,sex
0,0,半野雅,176,65,20.983988,1984年07月05日（38歳）,男性
1,1,竹森まりあ,162,43,16.384697,1996年11月26日（25歳）,女性
2,2,松尾紘太朗,183,76,22.694019,1997年09月16日（24歳）,男性
3,3,南部麻衣,157,48,19.473407,1995年12月31日（26歳）,女性
4,4,関隼汰,174,58,19.157088,1999年08月18日（22歳）,男性
...,...,...,...,...,...,...,...
1962,1962,池田朋子,155,38,15.816857,1987年11月20日（34歳）,女性
1963,1963,菊池銀河,175,635,207.346939,2002年07月15日（19歳）,男性
1964,1964,大高雄一郎,180,72,22.222222,1987年12月18日（34歳）,男性
1965,1965,田中光,181,67,20.451146,1982年02月13日（40歳）,男性


In [91]:
df.shape

(1967, 7)

In [92]:
print(df[df["old"].isnull()])

      id   name  height  weight         bmi  old  sex
206  206   榎本実里     160     418  163.281250  NaN  NaN
207  207  山下みなみ     172      62   20.957274  NaN  NaN
208  208   市岡清里     160      51   19.921875  NaN  NaN
209  209   長代和昭     167      72   25.816630  NaN  NaN
210  210   長田陽行     173      56   18.710949  NaN  NaN
211  211   高島草太     169      53   18.556773  NaN  NaN
212  212   野地禎人     177      58   18.513199  NaN  NaN
213  213   平井早紀     153      48   20.504934  NaN  NaN
214  214  永井絵梨沙     155      50   20.811655  NaN  NaN
215  215    川畑蒿     175      60   19.591837  NaN  NaN
962  962   RENA     160      50   19.531250  NaN  NaN


In [93]:
df = df.dropna()
df.shape

(1956, 7)

In [94]:
print(df.duplicated(subset="name").sum())
print(df[df.duplicated(subset="name")])

3
        id   name  height  weight        bmi               old sex
934    934     咲良     145      30  14.268728  2006年12月22日（15歳）  女性
1251  1251  五十嵐小夏     157      45  18.256319  2003年06月13日（19歳）  女性
1308  1308   近藤美鈴     164      45  16.731112  1995年05月20日（27歳）  女性


確認したところ同姓同名で同一人物ではなかったため削除しない

In [95]:
dst_old = []
for i in df["old"]:
    o = int(i.split("（")[1].split("歳")[0])
    dst_old.append(o)
df.loc[:, "old"] = dst_old
df

c:\Users\MK\Anaconda3\envs\tensorflow_gpu\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,id,name,height,weight,bmi,old,sex
0,0,半野雅,176,65,20.983988,38,男性
1,1,竹森まりあ,162,43,16.384697,25,女性
2,2,松尾紘太朗,183,76,22.694019,24,男性
3,3,南部麻衣,157,48,19.473407,26,女性
4,4,関隼汰,174,58,19.157088,22,男性
...,...,...,...,...,...,...,...
1962,1962,池田朋子,155,38,15.816857,34,女性
1963,1963,菊池銀河,175,635,207.346939,19,男性
1964,1964,大高雄一郎,180,72,22.222222,34,男性
1965,1965,田中光,181,67,20.451146,40,男性


In [96]:
df.drop(["name"], axis=1, inplace=True)
df

c:\Users\MK\Anaconda3\envs\tensorflow_gpu\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,id,height,weight,bmi,old,sex
0,0,176,65,20.983988,38,男性
1,1,162,43,16.384697,25,女性
2,2,183,76,22.694019,24,男性
3,3,157,48,19.473407,26,女性
4,4,174,58,19.157088,22,男性
...,...,...,...,...,...,...
1962,1962,155,38,15.816857,34,女性
1963,1963,175,635,207.346939,19,男性
1964,1964,180,72,22.222222,34,男性
1965,1965,181,67,20.451146,40,男性


In [97]:
df = pd.get_dummies(df, columns=["sex"])
df

,id,height,weight,bmi,old,sex_女性,sex_男性
0,0,176,65,20.983988,38,0,1
1,1,162,43,16.384697,25,1,0
2,2,183,76,22.694019,24,0,1
3,3,157,48,19.473407,26,1,0
4,4,174,58,19.157088,22,0,1
...,...,...,...,...,...,...,...
1962,1962,155,38,15.816857,34,1,0
1963,1963,175,635,207.346939,19,0,1
1964,1964,180,72,22.222222,34,0,1
1965,1965,181,67,20.451146,40,0,1


In [99]:
df.columns = ["id", "height", "weight", "bmi", "old", "female", "male"]
df.to_csv("talents_dataset/combine.csv", mode="w", index=False)